## 概要

- `gemini`を使用して、論文のアブストラクトの評価を行うスクリプト
- `gemini`の出力は JSON 形式に固定
- `rules`で評価指標を指定
- ``で用語等の定義を指定


# setting


In [144]:
# 初期設定
import pathlib
import textwrap
import google.generativeai as genai
import os
from dotenv import load_dotenv
from IPython.display import display
from IPython.display import Markdown
import pandas as pd
import time
import json
from tqdm import tqdm
from random import randint

In [145]:
# システムプロンプト
system_instructions = """
You will judge whether a given abstract meets the specified evaluation metrics based on the given criteria. The following is the prerequisite knowledge used when evaluating abstracts.

[## Prerequisites

**Proposed Method**: A newly conceived approach or method to solve a specific problem. This includes improvements to existing methods. It encompasses the steps and operations of the method.

- **Example 1 (New Algorithm):** Developed a new algorithm to solve traffic congestion by adjusting traffic volume and signal timing in real-time. This algorithm inputs past traffic data and current traffic conditions into a machine learning model to output optimal signal control patterns. The proposed method includes the algorithm design, creation of learning data, parameter tuning method, and the procedure for applying it to actual traffic signals.
- **Example 2 (Improvement of Existing Method):** Added a new preprocessing step to the existing image recognition model to improve accuracy. This preprocessing involves noise removal and contrast adjustment. The proposed method includes the specific steps of this preprocessing and how it can be integrated into image recognition models.

**Dataset**: A collection of data used for experiments or analysis.

- **Example 1 (Image Data):** A dataset consisting of 10,000 cat images and 10,000 dog images. Each image is labeled with breed, fur color, and posture.
- **Example 2 (Text Data):** A dataset of 1 million tweets. Each tweet includes posting date and time, user ID, content, and hashtags.

**Experimental Equipment**: Physical devices or facilities used in an experiment. Software is not included.

- **Example 1:** EEG measurement device, electrodes, amplifier
- **Example 2:** Beakers, flasks, electronic balance for observing chemical reactions

**Data Collection Method**: The method of obtaining data that constitutes the dataset.

- **Example 1 (Survey):** Created an online survey form and recruited participants through social media to collect responses.
- **Example 2 (Sensor Measurement):** Installed temperature and humidity sensors to automatically record data every hour.

**Evaluation Metrics**: Specific measures to assess the performance or effectiveness of the proposed method or experimental results.

- **Example 1 (Image Classification Accuracy):** Percentage of correctly classified images
- **Example 2 (Algorithm Execution Speed):** Time taken to complete a specific task

**Experimental Results/Findings**: Specific data obtained from the experiment and the insights derived from that data.

- **Example 1:** When using proposed method A, image classification accuracy was 95%. In contrast, existing method B achieved 80%. This result shows that proposed method A achieves higher classification accuracy than existing method B.
- **Example 2:** Mice administered drug X showed significantly larger tumor reduction compared to the control group (p < 0.05). This suggests that drug X has anti-tumor effects.

**Research Prospects**: Description of future research directions or potential applications of research findings.

- **Example 1:** Future research will need to verify the performance of the proposed method using larger datasets. We will also investigate potential applications in other fields.
- **Example 2:** It is expected that improving the algorithm developed in this research could enable the construction of faster and more energy-efficient systems.

**Research Background**: The context that gave rise to the research topic, explaining why the topic is important and why research is necessary. Describes social needs, academic challenges, or unresolved problems.

- **Example:** "In recent years, abnormal weather events have become frequent due to global warming. Particularly, damage from concentrated heavy rainfall has become serious, making the strengthening of disaster prevention measures urgent. However, existing disaster prevention systems have low prediction accuracy and difficulty in prompt response."

**Overview of Previous Research and Its Relevance**: Introduces existing research and clarifies its relationship with the current research. Explains the achievements and limitations of previous research, and the novelty and contribution of the current research.

- **Example:** "Conventional concentrated heavy rainfall prediction systems primarily relied on weather radar data. However, these radar data have low spatial resolution and cannot accurately capture localized heavy rainfall. This research proposes a method to improve prediction accuracy by integrating radar data with ground-based rainfall gauge data and information from social media."

**Research Objective**: Clearly describes what the research aims to achieve. Specifies concrete goals or challenges to be solved.

- **Example:** "The objective of this research is to improve the accuracy of concentrated heavy rainfall prediction by comprehensively utilizing diverse data sources."

**Result Analysis Method**: Explains the methods used to analyze experimental results. Describes specific method names and procedures such as statistical methods, machine learning techniques, or simulations.

- **Example:** "To evaluate prediction accuracy, RMSE (Root Mean Squared Error) was used. To compare the performance of the proposed and conventional methods, a t-test was used to verify statistical significance."

**Statistical Trends**: Describes the statistical characteristics of the data. Explains data distribution and trends using statistical measures such as mean, median, standard deviation, correlation coefficient.

- **Example:** "When using the proposed method, RMSE decreased by 20% compared to the conventional method. Additionally, the correlation coefficient between prediction results and actual measurements was 0.85, indicating a high correlation."

**Discussion**: Interprets experimental results, discussing their meaning and significance. Explores reasons for obtaining certain results, causes of unexpected findings, and research limitations.

- **Example:** "The proposed method showed improved prediction accuracy compared to conventional methods. This is likely because integrating diverse data sources allowed more accurate capture of localized heavy rainfall. However, this research focused only on urban areas, and the applicability to areas with complex topography remains a future challenge."

**Conclusion**: Summarizes the entire research and briefly describes the findings. Describes the degree of achieving research objectives, main results, and future prospects. "Code has been made publicly available." is not a conclusion.

- **Example:** "This research proposed a heavy rainfall prediction system that comprehensively utilizes diverse data sources and verified its effectiveness. Experimental results confirmed that the proposed method improves prediction accuracy compared to conventional methods. Future research will investigate applicability to areas with complex topography."

**Comparing Main Results with Traditional Thinking**: Comparing the achievements of the proposed method with existing research and methods, clearly demonstrating its advantages and novelty. Qualitative comparisons are also effective.

- **Example:** "Conventional heavy rainfall prediction systems, relying solely on weather radar data, found it difficult to predict localized heavy rainfall. In contrast, the proposed system, by utilizing rainfall gauge data and social media information in addition to radar data, improved localized heavy rainfall prediction accuracy by 20%. Moreover, while existing systems required hours for prediction, the proposed system enables real-time prediction."]

---
# Instruction

1. **Section Decomposition**:
    Systematically divide the abstract into the following predefined sections:
    - Research Background
    - Prior Research
    - Research Purpose
    - Proposed Method
    - Dataset
    - Experimental Apparatus
    - Data Collection Method
    - Experimental Evaluation Metrics
    - Experimental Results
    - Fingings from the results
    - Result Analysis Method
    - Statistical Trends of Results
    - Quantitative Results
    - Qualitative Results
    - Discussion
    - Conclusion
    - Research Prospects

2. **Evaluation Format**:
    Provide a JSON response that evaluates whether the abstract meets specific criteria for each section.

3. **Evaluation Rules**:
    - If a section does not exist in the abstract, represent it as a blank list.
    - For evaluation metrics sections:
     * Answer with "no" for whether something is written or not

---
Use this JSON schema:

**JSON schema**:
{
    "results": [
        {
            "separated_abstract": ["string", ...],
            "rules": ["yes" or "no", ...]
        }
    ]
}

**Example Response**:
{
    "results": [
        {
            "separated_abstract": [
                "This research investigates the impact of climate change on agriculture.",
                "We propose a novel machine learning model to predict crop yields.",
                "Our results show a significant decrease in yields in the next decade.",
                "The findings suggest the urgent need for adaptation strategies."
            ],
            "rules": ["yes", "no", "yes", "yes", "no", "yes", "yes", "no"]
        }
    ]
}
"""

In [152]:
# 評価指標
rules = """
Rules:
1. Research background is written
2. Research background is shown in one sentence
3. Research background is shown in two sentences
4. Overview of previous research and its relevance is written
5. Research objective is written
6. Research objective is in the first sentence of the abstract
7. Research objective is within the first three sentences of the abstract
8. Research objective is written in present tense
9. Research objective is written in past tense
10. The necessity of achieving the research objective is written
11. Proposed research method is written
12. Proposed research method is written in past tense
13. Explanation of dataset content exists
14. Experimental equipment is explained
15. Data collection method is written
16. Experimental evaluation metrics are written
17. Experimental results are written
18. Findings from the results are written
19. Experimental results and findings are in present tense
20. Experimental results and findings are in past tense
21. Result analysis method is written
22. Statistical trends are written
23. Quantitative results are written
24. Qualitative results are written
25. Discussion is written
26. Conclusion is written
27. Discussion is in present tense
28. Conclusion is in present tense
29. Main results are explained while comparing with traditional thinking
30. Research prospects are written in the last 1-2 sentences
31. Abbreviations are included (except when first explained)
32. Written in active voice
33. Use of I or We as the subject
"""

In [147]:
# モデルのインスタンスを作成
genai.configure(api_key=os.getenv("API_KEY"))
model = genai.GenerativeModel(
    model_name = "gemini-1.5-flash",
    system_instruction = system_instructions,
    generation_config={
                        "response_mime_type": "application/json",
                        "temperature": 0,
                        "top_p": 0,
                    }
)

## 関数の定義


In [148]:
def create_user_message(abstract):
    return f"""
    Abstract: {abstract}
    
    {rules}
    """

# geminiを使った回答の生成
def generate_response(model, abstract):
    response = model.generate_content(create_user_message(abstract))
    return response

# responseカラムを解析してrule1, rule2, rule3に分解
def parse_response(response):
    if pd.isna(response):
        return {}
    try:
        # JSON形式をデコード
        parsed = json.loads(response)
        if isinstance(parsed, list):  # リストの場合
            if isinstance(parsed[0], dict) and "rules" in parsed[0]:
                rules = parsed[0]["rules"]
            else:
                rules = []
        elif isinstance(parsed, dict):  # 辞書の場合
            if "results" in parsed:  # "results"キーがある場合
                rules = parsed["results"][0].get("rules", [])
            else:  # "rules"キーが直接ある場合
                rules = parsed.get("rules", [])
        else:
            rules = []
        # ルールを辞書形式で返す
        return {f"rule{i+1}": rule for i, rule in enumerate(rules)}
    except json.JSONDecodeError:
        return {}


# Gemini を使用した回答の生成


In [149]:
# 基本パスを指定
base_input_path = "../data/csv"
base_output_path = "../data/results"
fields = ["Biochemistry_Molecular_Biology", "Chemistry", "Engineering", "Materials_Science", "Physics"]

In [150]:
# 処理する分野とhigh/lowを指定
selected_field = fields[0]  # 分野を指定
selected_citation = "high"  # "high" または "low" を指定

In [151]:
# 処理対象のフォルダと出力先フォルダ
field_input_path = os.path.join(base_input_path, selected_field)
field_output_path = os.path.join(base_output_path, selected_field)

# 出力フォルダが存在しない場合は作成
os.makedirs(field_output_path, exist_ok=True)

# 各フォルダ内の対象CSVファイルを取得（highまたはlowを含むファイルのみ）
csv_files = [
    f for f in os.listdir(field_input_path)
    if f.endswith(".csv") and selected_citation in f
]

# 各CSVファイルを処理
for file_name in csv_files:
    input_file = os.path.join(field_input_path, file_name)
    output_file = os.path.join(field_output_path, file_name)

    try:
        # CSVファイルを読み込み
        df = pd.read_csv(input_file, encoding="utf-8")
        print(f"データの読み込みに成功しました: {input_file}")
    except Exception as e:
        print(f"エラーが発生しました: {e}")
        continue

    # IDカラムの追加と並び替え
    df["ID"] = df.index  # インデックスをIDとして追加
    cols = ["ID"] + [col for col in df.columns if col != "ID"]  # IDを最初に移動
    df = df[cols]

    # アブストラクトを辞書形式リストとして作成
    abstracts = [
        {"abstract_id": row["ID"], "content": row["Abstract"]}
        for _, row in df.dropna(subset=["Abstract"]).iterrows()
    ]
    
    if not abstracts:
        print(f"アブストラクトが空のためスキップ: {file_name}")
        continue

    # モデルを使用して処理
    try:
        # tqdmを使用して進捗を表示
        raw_responses = []
        for i in tqdm(range(len(abstracts)), desc=f"Processing {file_name}"):
            abstract = abstracts[i]
            response = generate_response(model, abstract["content"])
            raw_responses.append({
                "abstract_id": abstract["abstract_id"],
                "response": response.text
            })
            time.sleep(5)  # インターバルを挿入

        print(f"処理完了: {file_name}")
    except Exception as e:
        print(f"処理中にエラーが発生しました: {e}")
        continue

    # 処理結果をDataFrameに変換
    try:
        # DataFrameに変換
        results_df = pd.DataFrame(raw_responses)

        # `response`をパースして新しいカラムを作成
        rules_df = results_df["response"].apply(parse_response).apply(pd.Series)

        # `abstract_id`にルールを結合
        results_df = pd.concat([results_df, rules_df], axis=1).drop(columns=["response"])

        # 元のDataFrameと評価結果を結合
        merged_df = df.merge(results_df, left_on="ID", right_on="abstract_id", how="left").drop(columns=["abstract_id"])

        # 指定カラムを除外
        columns_to_exclude = ["Publication Type", "Authors", "Title", "DOI"]
        filtered_df = merged_df.drop(columns=columns_to_exclude)

        # 保存処理
        filtered_df.to_csv(output_file, index=False, encoding="utf-8")
        print(f"結果を保存しました: {output_file}")
    except Exception as e:
        print(f"保存中にエラーが発生しました: {e}")

データの読み込みに成功しました: ../data/csv/Biochemistry_Molecular_Biology/Biochemistry_Molecular_Biology_high1000_5.csv


Processing Biochemistry_Molecular_Biology_high1000_5.csv:   0%|          | 0/100 [00:00<?, ?it/s]

Processing Biochemistry_Molecular_Biology_high1000_5.csv:   3%|▎         | 3/100 [00:25<13:32,  8.38s/it]


処理中にエラーが発生しました: 500 Unable to submit request because the service is temporarily unavailable.
データの読み込みに成功しました: ../data/csv/Biochemistry_Molecular_Biology/Biochemistry_Molecular_Biology_high1000_4.csv


Processing Biochemistry_Molecular_Biology_high1000_4.csv: 100%|██████████| 100/100 [8:06:43<00:00, 292.04s/it]   


...

処理完了: Biochemistry_Molecular_Biology_high1000_2.csv
結果を保存しました: ../data/results/Biochemistry_Molecular_Biology/Biochemistry_Molecular_Biology_high1000_2.csv
データの読み込みに成功しました: ../data/csv/Biochemistry_Molecular_Biology/Biochemistry_Molecular_Biology_high1000_1.csv


Processing Biochemistry_Molecular_Biology_high1000_1.csv:  39%|███▉      | 39/99 [30:19<46:39, 46.65s/it]


処理中にエラーが発生しました: 504 Deadline Exceeded
データの読み込みに成功しました: ../data/csv/Biochemistry_Molecular_Biology/Biochemistry_Molecular_Biology_high1000_10.csv


Processing Biochemistry_Molecular_Biology_high1000_10.csv:  26%|██▌       | 26/100 [03:33<10:07,  8.21s/it]


KeyboardInterrupt: 

In [ ]:
# file_name = "Biochemistry_Molecular_Biology_high1000_3"
# input_file = f"../data/csv/Bio/{file_name}.csv"
# output_file = f"../data/results/Bio/{file_name}.csv"

# # 読み込むデータの行数を指定
# num_rows_to_read = 5

# try:
#     # df = pd.read_csv(input_file, encoding="utf-8", nrows=num_rows_to_read)
#     df = pd.read_csv(input_file, encoding="utf-8")
#     print("データの読み込みに成功しました。")
# except Exception as e:
#     print(f"エラーが発生しました: {e}")

In [ ]:
# df["ID"] = df.index
# cols = ["ID"] + [col for col in df.columns if col != "ID"]
# df = df[cols]
# abstracts = [
#     {"abstract_id": row["ID"], "content": row["Abstract"]}
#     for _, row in df.dropna(subset=["Abstract"]).iterrows()
# ]

Gemini API を使用して回答を生成


In [ ]:
# %%time
# raw_responses = process_abstracts(model, abstracts, definition, instruction, rules)

In [ ]:
# # DataFrameに変換
# results_df = pd.DataFrame(raw_responses)

# # `response`をパースして新しいカラムを作成
# rules_df = results_df["response"].apply(parse_response).apply(pd.Series)

# # `abstract_id`にルールを結合
# results_df = pd.concat([results_df, rules_df], axis=1).drop(columns=["response"])

# # 元のDataFrameと評価結果を結合
# merged_df = df.merge(results_df, left_on="ID", right_on="abstract_id", how="left").drop(columns=["abstract_id"])


In [ ]:
# merged_df.head()

In [ ]:
# # 指定したカラムを除外
# columns_to_exclude = ["Publication Type", "Authors", "Title", "DOI"]
# filtered_df = merged_df.drop(columns=columns_to_exclude)

In [ ]:
# filtered_df.to_csv(output_file, index=False, encoding="utf-8")